In [ ]:
import pylsl, math, pickle, mne
import numpy as np
import time

In [ ]:
# Load Model
fname = 'model.sav'
model = pickle.load(open(fname, 'rb'))

# Load average reference projection
avg_proj = mne.read_proj('average-ref-proj.fif')

In [ ]:
plot_duration = 1  # how many seconds of data to show
update_interval = 60  # ms between screen updates
pull_interval = 500  # ms between each pull operation

streams = pylsl.resolve_streams()

for info in streams:
    if info.type() == 'EEG':
        eeg_stream = info

In [ ]:
inlet = pylsl.StreamInlet(eeg_stream, max_buflen=plot_duration, processing_flags=pylsl.proc_clocksync | pylsl.proc_dejitter)
name = eeg_stream.name()
channel_count = eeg_stream.channel_count()
bufsize = (2 * math.ceil(eeg_stream.nominal_srate() * plot_duration), eeg_stream.channel_count())
dtypes = [[], np.float32, np.float64, None, np.int32, np.int16, np.int8, np.int64]
buffer = np.empty(bufsize, dtype=dtypes[eeg_stream.channel_format()])
empty = np.array([])

In [ ]:
mne.set_log_level('ERROR')
ch_names = ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2', 'P3', 'Pz', 'P4', 'PO3', 'PO4', 'O1', 'Oz', 'O2']
sfreq = 125
eeg_info = mne.create_info(ch_names, sfreq, ch_types='eeg')


In [ ]:
# streams = resolve_stream('type', 'EEG')
# inlet = StreamInlet(streams[0])

while True:
    _, ts = inlet.pull_chunk(timeout=0.0, max_samples=buffer.shape[0], dest_obj=buffer)
    
    if ts:
        ts = np.asarray(ts)
        
        # Create MNE Raw object
        raw = mne.io.RawArray(np.nan_to_num(np.transpose(buffer / 1e6)), eeg_info)
        raw = raw.add_proj(avg_proj)
        raw = raw.filter(l_freq=1., h_freq=40., n_jobs=-1)

        y_predict = model.predict(np.expand_dims(raw.get_data()[:,-126:], axis=0))

        if y_predict[0] == 0:
            print("Rest")
        else:
            print("MI")
        
    time.sleep(1./250.)